In [4]:
import cv2
import numpy as np

def grab_cut(img):
    mask_img = np.zeros(img.shape[:2], np.uint8)  # 초기 마스크

    # grabcut에 사용할 임시 배열
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)

    rect = (74, 70, 420, 570-70)  # 명함 영역으로 가정
    cv2.grabCut(img, mask_img, rect, bgdModel, fgdModel, 3, cv2.GC_INIT_WITH_RECT)  # grabcut 실행
    mask_img = np.where((mask_img == 2) | (mask_img == 0), 0, 1).astype('uint8')  # 배경인 곳은 0, 그 외에는 1로 설정한 마스크를 만든다.
    img = img * mask_img[:, :, np.newaxis]  # 이미지에 새로운 마스크를 곱해 배경을 제외한다.

    background = img.copy()
    background[np.where((background >= [0, 0, 0]).all(axis=2))] = [0, 0, 0]

    img_grabcut = background + img

    cv2.imshow('grabcut', img_grabcut)
    cv2.waitKey(0)

    new_edged = edge_detection(img_grabcut)
    
    # 배경과 명함 경계가 모호한 경우, 적절한 전처리 및 윤곽선 추출 알고리즘을 적용하여 명함 영역을 추출한다.
    new_edged = cv2.cvtColor(new_edged, cv2.COLOR_GRAY2BGR)  # 채널 수 조정
    
    edged = edge_detection(new_edged)
    card_contour = contoursGrab(edged)
    
    # 명함 윤곽선을 이용하여 명함 영역만을 잘라낸다.
    if card_contour is not None:
        mask = np.zeros_like(img_grabcut)
        cv2.drawContours(mask, [card_contour], -1, (255, 255, 255), thickness=cv2.FILLED)
        img_grabcut = cv2.bitwise_and(img_grabcut, mask)
    
    cv2.imshow('edge detection', edged)
    cv2.waitKey(0)
    
    return img_grabcut

def edge_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 30, 100)
    return edges

def contoursGrab(edged):
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        card_contour = max(contours, key=cv2.contourArea)
        return card_contour
    else:
        return None
    
def remove_noise_and_approximate(rect):
    # 노이즈 제거
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(rect, cv2.MORPH_OPEN, kernel, iterations=2)

    # 작은 객체 제외
    gray = cv2.cvtColor(opening, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]

    # 직사각형 근사화
    approx_rects = []
    for cnt in filtered_contours:
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.03 * peri, True)
        if len(approx) == 4:
            approx_rects.append(approx)

    return approx_rects


# 예시 이미지
resized = cv2.imread("C:/Users/Yunji Noh/OneDrive/image_processing/data/BC16.jpg")

# grab_cut 함수 호출
result = grab_cut(resized)

# 결과 이미지 출력
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


# 노이즈 제거 및 직사각형 근사화
filtered_rects = remove_noise_and_approximate(result)

# 결과 출력
for rect in filtered_rects:
    cv2.drawContours(result, [rect], -1, (0, 255, 0), 2)
    
    # 원근변환을 위한 좌표 설정
    src_pts = np.float32(rect)
    dst_pts = np.float32([[400, 0],[0, 0],  [0, 200],[400, 200]])

    # 원근변환 행렬 계산
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # 원근변환 적용
    warped = cv2.warpPerspective(result, M, (400, 200), cv2.INTER_CUBIC)
    #sharp = cv2.filter2D(warped, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))

    cv2.imshow('Warped Result', warped)
    
    cv2.imshow('sharp Result', sharp)
    cv2.waitKey(0)
    cv2.destroyAllWindows()